In [1]:
!pip install -q kaggle

In [4]:
from google.colab import files

# Upload your kaggle.json
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shreshthvishwakarma","key":"1946174d78f71e30e48fd47ee8fb65d4"}'}

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download and unzip dataset
!kaggle datasets download -d vaibhavkumar11/hindi-english-parallel-corpus
!unzip -o hindi-english-parallel-corpus.zip


Dataset URL: https://www.kaggle.com/datasets/vaibhavkumar11/hindi-english-parallel-corpus
License(s): unknown
  0% 0.00/112M [00:00<?, ?B/s]
100% 112M/112M [00:00<00:00, 1.28GB/s]
Archive:  hindi-english-parallel-corpus.zip
  inflating: hindi_english_parallel.csv  


In [6]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"


In [2]:
!pip install torch==2.1.0 torchtext==0.16.0 --force-reinstall --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
import torch, time
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import GloVe, FastText, vocab
from collections import Counter
import pandas as pd, re, string
from torch.cuda.amp import autocast, GradScaler
from torchtext.data.metrics import bleu_score



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [8]:
def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub("\s+", " ", text).strip()
    return text

df = pd.read_csv("hindi_english_parallel.csv").dropna()
df = df.rename(columns={"English": "english", "Hindi": "hindi"})
df['english'] = df['english'].apply(clean_text)
df['hindi'] = df['hindi'].apply(clean_text)

en_tok = get_tokenizer("basic_english")
hi_tok = lambda x: x.split()
df['en_tokens'] = df['english'].apply(en_tok)
df['hi_tokens'] = df['hindi'].apply(hi_tok)


In [9]:
eng_counter = Counter([token for row in df['en_tokens'] for token in row])
hin_counter = Counter([token for row in df['hi_tokens'] for token in row])
eng_vocab = vocab(eng_counter, specials=["<pad>", "<unk>", "<sos>", "<eos>"])
hin_vocab = vocab(hin_counter, specials=["<pad>", "<unk>", "<sos>", "<eos>"])
eng_vocab.set_default_index(eng_vocab["<unk>"])
hin_vocab.set_default_index(hin_vocab["<unk>"])

PAD_IDX = eng_vocab["<pad>"]
SOS_IDX = hin_vocab["<sos>"]
EOS_IDX = hin_vocab["<eos>"]


In [10]:
glove = GloVe(name='6B', dim=300)
fasttext = FastText(language='hi')



.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                           
100%|█████████▉| 399999/400000 [00:46<00:00, 8679.46it/s]
.vector_cache/wiki.hi.vec: 417MB [00:05, 77.6MB/s]                          
100%|██████████| 158016/158016 [00:17<00:00, 8819.90it/s]


In [11]:
eng_embed = torch.zeros(len(eng_vocab), 300)
for w, i in eng_vocab.get_stoi().items():
    eng_embed[i] = glove[w] if w in glove.stoi else torch.randn(300)

hin_embed = torch.zeros(len(hin_vocab), 300)
for w, i in hin_vocab.get_stoi().items():
    hin_embed[i] = fasttext[w] if w in fasttext.stoi else torch.randn(300)


In [12]:
pairs = []
for _, row in df.iterrows():
    en = [eng_vocab[t] for t in row['en_tokens']]
    hi = [hin_vocab["<sos>"]] + [hin_vocab[t] for t in row['hi_tokens']] + [hin_vocab["<eos>"]]
    pairs.append((torch.tensor(en), torch.tensor(hi)))

pairs = pairs[:100000]


In [13]:
class MTDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
    def __len__(self):
      return len(self.pairs)
    def __getitem__(self, index):
      return self.pairs[index]


def collate_fn(batch):
    src, trg = zip(*batch)
    return pad_sequence(src, padding_value=PAD_IDX), pad_sequence(trg, padding_value=PAD_IDX)



loader = DataLoader(MTDataset(pairs), batch_size=8, shuffle=True, collate_fn=collate_fn)


In [14]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, embedding_matrix):
        super().__init__()
        self.embed = nn.Embedding.from_pretrained(embedding_matrix)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=1)
    def forward(self, x):
        embedded = self.embed(x)
        return self.lstm(embedded)

class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, embedding_matrix):
        super().__init__()
        self.embed = nn.Embedding.from_pretrained(embedding_matrix)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=1)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    def forward(self, input, hidden, cell):
        embedded = self.embed(input.unsqueeze(0))
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        return self.fc(output.squeeze(0)), hidden, cell


In [15]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    def forward(self, src, trg, teacher_forcing_ratio=0.75):
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        vocab_size = self.decoder.fc.out_features
        outputs = torch.zeros(max_len, batch_size, vocab_size).to(self.device)
        i, (hidden, cell) = self.encoder(src)
        input = trg[0, :]
        for t in range(1, max_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            top1 = output.argmax(1)
            if torch.rand(1).item()<teacher_forcing_ratio:
                input = trg[t]
            else:
                input = top1

        return outputs


In [16]:
HID_DIM = 128

model = Seq2Seq(Encoder(len(eng_vocab), 300, HID_DIM, eng_embed), Decoder(len(hin_vocab), 300, HID_DIM, hin_embed), device).to(device)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
scaler = GradScaler()


In [17]:
def train(model, loader, optimizer, criterion):
    model.train()
    epoch_loss = 0

    for eng, hin in loader:
        eng, hin = eng.to(device).long(), hin.to(device).long()

        optimizer.zero_grad()
        with autocast():
            output = model(eng, hin)
            loss = criterion(output[1:].reshape(-1, output.shape[-1]), hin[1:].reshape(-1))

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()

    return epoch_loss/len(loader)


In [18]:
def compute_bleu(model, dataset, n=50):
    model.eval()
    refs, hyps = [], []
    for i in range(n):
        src, trg = dataset[i]
        src = src.unsqueeze(1).to(device)
        with torch.no_grad():
            _, (h, c) = model.encoder(src)
            x = torch.tensor([SOS_IDX]).to(device)
            pred = []
            for _ in range(20):
                out, h, c = model.decoder(x, h, c)
                top1 = out.argmax(1).item()
                if top1 == EOS_IDX:
                    break
                pred.append(top1)
                x = torch.tensor([top1]).to(device)
        ref = [hin_vocab.lookup_token(t.item()) for t in trg[1:-1]]
        hyp = [hin_vocab.lookup_token(i) for i in pred]
        refs.append([ref])
        hyps.append(hyp)
    return bleu_score(hyps, refs)


In [19]:
# Emergency fix for numpy error in torch.randperm().numpy()
import torch

if not hasattr(torch.Tensor, "original_numpy"):
    torch.Tensor.original_numpy = torch.Tensor.numpy
    torch.Tensor.numpy = lambda self: self.detach().cpu().tolist()


In [20]:
for epoch in range(3):
    loss = train(model, loader, optimizer, criterion)
    bleu = compute_bleu(model, pairs, n=30)
    print(f"Epoch {epoch+1} ------ Loss: {loss:.6f} ---- BLEU: {bleu:.6f}")


Epoch 1 ------ Loss: 4.470495 ---- BLEU: 0.000000
Epoch 2 ------ Loss: 2.570403 ---- BLEU: 0.249307
Epoch 3 ------ Loss: 1.778311 ---- BLEU: 0.460397


In [21]:
class AttentionDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, embedding_matrix):
        super().__init__()
        self.embed = nn.Embedding.from_pretrained(embedding_matrix)
        self.lstm = nn.LSTM(embed_dim + hidden_dim, hidden_dim, num_layers=1)
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc = nn.Linear(hidden_dim * 2, vocab_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input, hidden, cell, encoder_outputs):
        input = input.unsqueeze(0)
        embedded = self.embed(input)

        hidden_last = hidden[-1]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        attn_scores = torch.bmm(encoder_outputs, hidden_last.unsqueeze(2)).squeeze(2)
        attn_weights = self.softmax(attn_scores)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).permute(1, 0, 2)

        lstm_input = torch.cat((embedded, context), dim=2)
        output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))

        output = output.squeeze(0)
        context = context.squeeze(0)
        prediction = self.fc(torch.cat((output, context), dim=1))
        return prediction, hidden, cell, attn_weights


In [22]:
class Seq2SeqAttention(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        vocab_size = self.decoder.fc.out_features
        outputs = torch.zeros(max_len, batch_size, vocab_size).to(self.device)

        encoder_outputs, (hidden, cell) = self.encoder(src)
        input = trg[0, :]

        for t in range(1, max_len):
            output, hidden, cell, _ = self.decoder(input, hidden, cell, encoder_outputs)
            outputs[t] = output
            top1 = output.argmax(1)
            input = trg[t] if torch.rand(1).item() < teacher_forcing_ratio else top1

        return outputs

In [23]:
scaler = GradScaler()

def train_attention(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for src, trg in loader:
        src, trg = src.to(device).long(), trg.to(device).long()
        optimizer.zero_grad()
        with autocast():
            output = model(src, trg)
            loss = criterion(output[1:].reshape(-1, output.shape[-1]), trg[1:].reshape(-1))
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
    return total_loss / len(loader)


In [24]:
def compute_bleu(model, pairs, n=30):
    model.eval()
    refs, hyps = [], []
    for i in range(n):
        src_tensor = pairs[i][0].unsqueeze(1).to(device)
        trg_tensor = pairs[i][1]
        output_tokens = []
        with torch.no_grad():
            enc_outs, (hidden, cell) = model.encoder(src_tensor)
            input = torch.tensor([hin_vocab['<sos>']], device=device)
            for _ in range(30):
                out, hidden, cell, _ = model.decoder(input, hidden, cell, enc_outs)
                top1 = out.argmax(1)
                if top1.item() == hin_vocab['<eos>']:
                    break
                output_tokens.append(hin_vocab.lookup_token(top1.item()))
                input = top1

        target_tokens = [hin_vocab.lookup_token(tok.item()) for tok in trg_tensor[1:-1]]
        hyps.append(output_tokens)
        refs.append([target_tokens])
    return bleu_score(hyps, refs)


In [25]:
HID_DIM = 128
model = Seq2SeqAttention(Encoder(len(eng_vocab), 300, HID_DIM, eng_embed), AttentionDecoder(len(hin_vocab), 300, HID_DIM, hin_embed), device).to(device)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)



In [ ]:
for epoch in range(3):
    loss = train_attention(model, loader, optimizer, criterion)
    bleu = compute_bleu(model, pairs, n=30)
    print(f"Epoch {epoch+1} --- Loss: {loss:.6f} --- BLEU: {bleu:.6f}")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

def translate_and_get_attention(model, src_tensor):
    model.eval()
    src_tensor = src_tensor.unsqueeze(1).to(device)
    attentions = []

    with torch.no_grad():
        encoder_outputs, (hidden, cell) = model.encoder(src_tensor)
        input = torch.tensor([hin_vocab['<sos>']], device=device)
        output_tokens = []

        for _ in range(20):
            output, hidden, cell, attn_weights = model.decoder(input, hidden, cell, encoder_outputs)
            top1 = output.argmax(1)
            if top1.item() == hin_vocab['<eos>']:
                break
            output_tokens.append(top1.item())
            attentions.append(attn_weights.squeeze(0).cpu().numpy())
            input = top1

    return output_tokens, np.array(attentions)


def plot_attention(input_sentence, output_tokens, attentions, index):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111)

    cax = ax.matshow(attentions, cmap='viridis')
    fig.colorbar(cax)

    ax.set_xticklabels([''] + input_sentence, rotation=45)
    ax.set_yticklabels([''] + output_tokens)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.title(f"Example {index+1}")
    plt.xlabel("Input Sentence")
    plt.ylabel("Output Prediction")
    plt.tight_layout()
    plt.show()


In [ ]:
for i in range(3):
    src_tensor, trg_tensor = pairs[i+50]
    output_indices, attn_weights = translate_and_get_attention(model, src_tensor)

    input_tokens = [eng_vocab.lookup_token(tok.item()) for tok in src_tensor]
    output_tokens = [hin_vocab.lookup_token(tok) for tok in output_indices]

    plot_attention(input_tokens, output_tokens, attn_weights, i)
